<a href="https://colab.research.google.com/github/gremlin97/Hard-Debias/blob/main/HardDebiasGWE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AdamMiyauchi/CSE575-Project.git

Cloning into 'CSE575-Project'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 33 (delta 9), reused 30 (delta 6), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [2]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
!ls

CSE575-Project	sample_data


In [4]:
df = pd.concat([
    pd.read_csv('CSE575-Project/data/2018-E-c-En-train.tsv', delimiter='\t', encoding='utf-8', na_values='NONE'),
    pd.read_csv('CSE575-Project/data/2018-E-c-En-test.tsv', delimiter='\t', encoding='utf-8', na_values='NONE'),
    pd.read_csv('CSE575-Project/data/2018-E-c-En-dev.tsv', delimiter='\t', encoding='utf-8', na_values='NONE'),
])
df = (df
    .drop('ID', axis=1)
    .rename(columns={'Tweet': 'text'})
)

df['emotion'] = df.iloc[:, 1:].idxmax(1)
df = df[df['emotion'].isin(['anger', 'fear', 'joy', 'sadness'])]
df = df[['text', 'emotion']]

print('Total number of samples: ', df.shape)
df.sample(10)
print('Number of samples in each emotion class: ')
df['emotion'].value_counts()

Total number of samples:  (5847, 2)


,text,emotion
2594,"GKN so lively as well, mad quick",joy
6403,@britishairways term 5 security queues at arri...,anger
3603,The #Texans should never play a prime time gam...,anger
6238,@EducatedNPetty white pricks that were laughin...,anger
4543,@Roger_Dagenham Aye. There's a brief window in...,anger
4852,last nigt i dreamt \nthat somebody loved me\nn...,joy
2823,. @KotakBankLtd @udaykotak official got in tou...,anger
871,@ProfessorF @Mediaite @law_newz Childish tempe...,anger
2374,Food that gets delivered 😍🙌🏻,joy
4013,@simmy_hanley @Schrise also a #delight,joy


Number of samples in each emotion class: 


anger      3077
joy        1938
fear        629
sadness     203
Name: emotion, dtype: int64

In [5]:
!pip install emoji
import nltk
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 5.6 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=50663b10f92f9fbb595186803bc7ec6d50e7cda083c8dabfe69414e4f5faca20
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import string
import nltk
from nltk.corpus import stopwords
import emoji
import re

def remove_punctuation(text):
    ans = ''
    for i in text:     
        if i not in string.punctuation:
            ans += i    
    return ans

def remove_stopwords(text):
    return ' '.join([word for word in text.split(" ") if word not in set(stopwords.words('english'))])

def remove_emoji(text):
    return emoji.demojize(text)

def remove_url(text):
    return re.sub(r"http:\S+",'', text)

def preprocess_text(text):
    return remove_punctuation(remove_stopwords(remove_emoji(remove_url(text))))

def emotion_to_int(emotion):
    if emotion == 'sadness': return 0
    elif emotion == 'anger': return 1
    elif emotion == 'joy': return 2
    elif emotion == 'fear': return 3
    # else: return 4  # neutral

def int_to_emotion(emotion_int):
    if emotion_int == 0: return 'sadness'
    elif emotion_int == 1: return 'anger'
    elif emotion_int == 2: return 'joy'
    elif emotion_int == 3: return 'fear'
    # else: return 'neutral'  # neutral



df['text'] = df['text'].apply(lambda x: preprocess_text(x))

labels = ['sadness', 'anger', 'joy', 'fear']
df['emotion'] = df['emotion'].apply(lambda x: emotion_to_int(x))

In [7]:
print('Total number of samples: ', df.shape)
df.sample(10)
print('Number of samples in each emotion class: ')
df['emotion'].value_counts()

Total number of samples:  (5847, 2)


,text,emotion
3594,Chick told I look like Childish Gambino insult...,1
6693,SAHARTHERAPPER I unfollowed without hesitation...,1
380,niceoneWern AlottaPain gleesome threesome,2
1618,TheNiceBot WSJNordics You make world joyful pl...,2
1890,Its easy fall asleep class hard bed True sober...,1
5734,Never switched up stayed person since day one,2
1406,aroseblush Hello nThe bigger bully crocodile t...,1
4203,I get soooo nervous actually attractive guy tr...,3
348,Im shy girlfacewithrollingeyes,0
2394,jccrocker CharlotteProtest u wait 4 facts vide...,0


Number of samples in each emotion class: 


1    3077
2    1938
3     629
0     203
Name: emotion, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

X, y = df['text'], df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [ ]:
%cd bert-sklearn/
!ls

/content/bert-sklearn
bert_sklearn		       glue_examples  other_examples  tests
demo.ipynb		       LICENSE	      README.md
demo_tuning_hyperparams.ipynb  Options.md     setup.py


In [ ]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 132 kB 26.3 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 9.9 MB 66.6 MB/s 
     |████████████████████████████████| 140 kB 68.2 MB/s 
     |████████████████████████████████| 127 kB 70.0 MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54248 sha256=b5a45ad01794a27c26563298a94171ac186e7e6fca5d59684112e26496d8ff05
  Stored in directory:

In [ ]:
from bert_sklearn import BertClassifier     
from bert_sklearn import load_model
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

baseline_BERT = BertClassifier()

baseline_BERT.num_mlp_layers = 1
baseline_BERT.epochs = 5 

baseline_BERT.fit(X_train, y_train)
baseline_BERT_df_pred = baseline_BERT.predict(X_test)

savefile='/content/CSE575-Project/models/baselineBERT.bin'
baseline_BERT.save(savefile)

Building sklearn text classifier...
Loading bert-base-uncased model...
Using mlp with D=768,H=500,K=4,n=1
Loading Pytorch checkpoint

train data size: 3526, validation data size: 391



Training  :   0%|          | 0/110 [00:00<?, ?it/s]

Validating:   0%|          | 0/49 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.8322, Val loss: 0.7354, Val accy: 74.42%



Training  :   0%|          | 0/110 [00:00<?, ?it/s]

...

Training  :   0%|          | 0/110 [00:00<?, ?it/s]

Validating:   0%|          | 0/49 [00:00<?, ?it/s]


Epoch 5, Train loss: 0.0655, Val loss: 0.6034, Val accy: 83.12%



BertClassifier(do_lower_case=True, epochs=5, label_list=array([0, 1, 2, 3]),
               num_mlp_layers=1)

Predicting:   0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:
!ls '/content/CSE575-Project/models'

baselineBERT_goemotions.bin	    dataset_debias_BERT_SemEval2018TaskEC.bin
baselineBERT_SemEval2018TaskEC.bin


In [9]:
from nltk.test.gensim_fixt import setup_module

In [10]:
setup_module()

In [11]:
import gensim

In [12]:
df_text = df['text']

In [86]:
len(df_text)

5847

In [13]:
text = []
for x in df_text:
  temp = x +'.'
  text.append(temp)

In [87]:
text[0:10]

['Whatever decide make sure makes happy.',
 'MaxKellerman  also helps majority NFL coaching inept Some Bill OBriens play calling wow  GOPATS.',
 'Accept challenges literally even feel exhilaration victory  George S Patton dogface.',
 'My roommate okay cant spell autocorrect terrible firstworldprobs.',
 'No thats cute Atsu probably shy photos cherry helped uwu.',
 'Rooneys fucking untouchable he Been fucking dreadful again depay looked decentishtonight.',
 'BossUpJaee pussy weak I heard stfu bitch  You got threaten pregnant .',
 'And hard dance devil backnSo shake off.',
 'Tiller breezy collab album Rapping singing prolly fire.',
 'SO girl hit carnot get lucky w scratch also spared wrath sleep deprived Kaitupsidedownface.']

In [82]:
len(text)

5847

In [15]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
nltk.download('punkt')
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec

text_tk = []

for i in df_text:
  temp = []
  for j in word_tokenize(i):
    temp.append(j.lower())
  text_tk.append(temp)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
text_tk[0:10]

[['whatever', 'decide', 'make', 'sure', 'makes', 'happy'],
 ['maxkellerman',
  'also',
  'helps',
  'majority',
  'nfl',
  'coaching',
  'inept',
  'some',
  'bill',
  'obriens',
  'play',
  'calling',
  'wow',
  'gopats'],
 ['accept',
  'challenges',
  'literally',
  'even',
  'feel',
  'exhilaration',
  'victory',
  'george',
  's',
  'patton',
  'dogface'],
 ['my',
  'roommate',
  'okay',
  'cant',
  'spell',
  'autocorrect',
  'terrible',
  'firstworldprobs'],
 ['no',
  'thats',
  'cute',
  'atsu',
  'probably',
  'shy',
  'photos',
  'cherry',
  'helped',
  'uwu'],
 ['rooneys',
  'fucking',
  'untouchable',
  'he',
  'been',
  'fucking',
  'dreadful',
  'again',
  'depay',
  'looked',
  'decentishtonight'],
 ['bossupjaee',
  'pussy',
  'weak',
  'i',
  'heard',
  'stfu',
  'bitch',
  'you',
  'got',
  'threaten',
  'pregnant'],
 ['and', 'hard', 'dance', 'devil', 'backnso', 'shake', 'off'],
 ['tiller',
  'breezy',
  'collab',
  'album',
  'rapping',
  'singing',
  'prolly',
  'fire

In [17]:
wes = gensim.models.Word2Vec(text_tk, min_count=5, size=300)

In [18]:
wes.save('dsembed')

In [19]:
print(wes)

Word2Vec(vocab=1999, size=300, alpha=0.025)


In [20]:
len(wes.wv['he'])

300

In [21]:
len(wes['he'])

300

In [92]:
list(wes.wv.vocab.items())[0:10]

[('whatever', <gensim.models.keyedvectors.Vocab at 0x7ff8b91d4490>),
 ('decide', <gensim.models.keyedvectors.Vocab at 0x7ff88a5d9950>),
 ('make', <gensim.models.keyedvectors.Vocab at 0x7ff89d973690>),
 ('sure', <gensim.models.keyedvectors.Vocab at 0x7ff8a7743290>),
 ('makes', <gensim.models.keyedvectors.Vocab at 0x7ff88d4bc710>),
 ('happy', <gensim.models.keyedvectors.Vocab at 0x7ff88a4c4150>),
 ('also', <gensim.models.keyedvectors.Vocab at 0x7ff88a4c4090>),
 ('majority', <gensim.models.keyedvectors.Vocab at 0x7ff88a4c4310>),
 ('coaching', <gensim.models.keyedvectors.Vocab at 0x7ff88d4bc2d0>),
 ('some', <gensim.models.keyedvectors.Vocab at 0x7ff88a4c4210>)]

In [23]:
!ls

CSE575-Project	dsembed  sample_data


In [24]:
!git clone https://github.com/tolga-b/debiaswe

Cloning into 'debiaswe'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (95/95), done.


In [25]:
%cd debiaswe

/content/debiaswe


In [26]:
!ls

data  debiaswe	embeddings  LICENSE  README.md	tutorial_example1.ipynb


In [27]:
%cd debiaswe
!ls 

/content/debiaswe/debiaswe
data.py  debias.py  __init__.py  learn_gender_specific.py  we.py


In [28]:
from debiaswe import debiaswe as dwe
from debiaswe.debiaswe import we
from debiaswe.debiaswe.we import WordEmbedding
from debiaswe.debiaswe.data import load_professions

In [29]:
em = wes

In [30]:
type(em)

gensim.models.word2vec.Word2Vec

In [31]:
wes.wv.save_word2vec_format('embeddingtext1.txt')

In [32]:
!pwd

/content/debiaswe/debiaswe


In [33]:
!cd debiaswe

/bin/bash: line 0: cd: debiaswe: No such file or directory


In [34]:
model = wes.wv.load_word2vec_format('/content/debiaswe/debiaswe/embeddingtext1.txt')

In [35]:
print(model)

In [41]:
nem = WordEmbedding('/content/debiaswe/debiaswe/embeddingtext1.txt')

*** Reading data from /content/debiaswe/debiaswe/embeddingtext1.txt
(1999, 300)
1999 words of dimension 300 : i, im, like, amp, ..., strike, cruel, michael, arsehole
1999 words of dimension 300 : i, im, like, amp, ..., strike, cruel, michael, arsehole


In [93]:
nem.words[0:10]

['i', 'im', 'like', 'amp', 'get', 'you', 'happy', 'people', 'the', 'one']

In [43]:
professions = load_professions()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


In [44]:
profession_words = ['acquaintance',
 'actor',
 'actress',
 'adjunct_professor',
 'administrator',
 'adventurer',
 'advocate',
 'aide',
 'alderman',
 'alter_ego',
 'ambassador',
 'analyst',
 'anthropologist',
 'archaeologist',
 'archbishop',
 'architect',
 'artist',
 'artiste',
 'assassin',
 'assistant_professor',
 'associate_dean',
 'associate_professor',
 'astronaut',
 'astronomer',
 'athlete',
 'athletic_director',
 'attorney',
 'author',
 'baker',
 'ballerina',
 'ballplayer',
 'banker',
 'barber',
 'baron',
 'barrister',
 'bartender',
 'biologist',
 'bishop',
 'bodyguard',
 'bookkeeper',
 'boss',
 'boxer',
 'broadcaster',
 'broker',
 'bureaucrat',
 'businessman',
 'businesswoman',
 'butcher',
 'butler',
 'cab_driver',
 'cabbie',
 'cameraman',
 'campaigner',
 'captain',
 'cardiologist',
 'caretaker',
 'carpenter',
 'cartoonist',
 'cellist',
 'chancellor',
 'chaplain',
 'character',
 'chef',
 'chemist',
 'choreographer',
 'cinematographer',
 'citizen',
 'civil_servant',
 'cleric',
 'clerk',
 'coach',
 'collector',
 'colonel',
 'columnist',
 'comedian',
 'comic',
 'commander',
 'commentator',
 'commissioner',
 'composer',
 'conductor',
 'confesses',
 'congressman',
 'constable',
 'consultant',
 'cop',
 'correspondent',
 'councilman',
 'councilor',
 'counselor',
 'critic',
 'crooner',
 'crusader',
 'curator',
 'custodian',
 'dad',
 'dancer',
 'dean',
 'dentist',
 'deputy',
 'dermatologist',
 'detective',
 'diplomat',
 'director',
 'disc_jockey',
 'doctor',
 'doctoral_student',
 'drug_addict',
 'drummer',
 'economics_professor',
 'economist',
 'editor',
 'educator',
 'electrician',
 'employee',
 'entertainer',
 'entrepreneur',
 'environmentalist',
 'envoy',
 'epidemiologist',
 'evangelist',
 'farmer',
 'fashion_designer',
 'fighter_pilot',
 'filmmaker',
 'financier',
 'firebrand',
 'firefighter',
 'fireman',
 'fisherman',
 'footballer',
 'foreman',
 'freelance_writer',
 'gangster',
 'gardener',
 'geologist',
 'goalkeeper',
 'graphic_designer',
 'guidance_counselor',
 'guitarist',
 'hairdresser',
 'handyman',
 'headmaster',
 'historian',
 'hitman',
 'homemaker',
 'hooker',
 'housekeeper',
 'housewife',
 'illustrator',
 'industrialist',
 'infielder',
 'inspector',
 'instructor',
 'interior_designer',
 'inventor',
 'investigator',
 'investment_banker',
 'janitor',
 'jeweler',
 'journalist',
 'judge',
 'jurist',
 'laborer',
 'landlord',
 'lawmaker',
 'lawyer',
 'lecturer',
 'legislator',
 'librarian',
 'lieutenant',
 'lifeguard',
 'lyricist',
 'maestro',
 'magician',
 'magistrate',
 'maid',
 'major_leaguer',
 'manager',
 'marksman',
 'marshal',
 'mathematician',
 'mechanic',
 'mediator',
 'medic',
 'midfielder',
 'minister',
 'missionary',
 'mobster',
 'monk',
 'musician',
 'nanny',
 'narrator',
 'naturalist',
 'negotiator',
 'neurologist',
 'neurosurgeon',
 'novelist',
 'nun',
 'nurse',
 'observer',
 'officer',
 'organist',
 'painter',
 'paralegal',
 'parishioner',
 'parliamentarian',
 'pastor',
 'pathologist',
 'patrolman',
 'pediatrician',
 'performer',
 'pharmacist',
 'philanthropist',
 'philosopher',
 'photographer',
 'photojournalist',
 'physician',
 'physicist',
 'pianist',
 'planner',
 'plastic_surgeon',
 'playwright',
 'plumber',
 'poet',
 'policeman',
 'politician',
 'pollster',
 'preacher',
 'president',
 'priest',
 'principal',
 'prisoner',
 'professor',
 'professor_emeritus',
 'programmer',
 'promoter',
 'proprietor',
 'prosecutor',
 'protagonist',
 'protege',
 'protester',
 'provost',
 'psychiatrist',
 'psychologist',
 'publicist',
 'pundit',
 'rabbi',
 'radiologist',
 'ranger',
 'realtor',
 'receptionist',
 'registered_nurse',
 'researcher',
 'restaurateur',
 'sailor',
 'saint',
 'salesman',
 'saxophonist',
 'scholar',
 'scientist',
 'screenwriter',
 'sculptor',
 'secretary',
 'senator',
 'sergeant',
 'servant',
 'serviceman',
 'sheriff_deputy',
 'shopkeeper',
 'singer',
 'singer_songwriter',
 'skipper',
 'socialite',
 'sociologist',
 'soft_spoken',
 'soldier',
 'solicitor',
 'solicitor_general',
 'soloist',
 'sportsman',
 'sportswriter',
 'statesman',
 'steward',
 'stockbroker',
 'strategist',
 'student',
 'stylist',
 'substitute',
 'superintendent',
 'surgeon',
 'surveyor',
 'swimmer',
 'taxi_driver',
 'teacher',
 'technician',
 'teenager',
 'therapist',
 'trader',
 'treasurer',
 'trooper',
 'trucker',
 'trumpeter',
 'tutor',
 'tycoon',
 'undersecretary',
 'understudy',
 'valedictorian',
 'vice_chancellor',
 'violinist',
 'vocalist',
 'waiter',
 'waitress',
 'warden',
 'warrior',
 'welder',
 'worker',
 'wrestler',
 'writer']

In [45]:
res_i = list(set(profession_words) & set(nem.words))

In [46]:
res_i

['writer',
 'teacher',
 'coach',
 'author',
 'student',
 'character',
 'doctor',
 'dad',
 'president']

In [47]:
v_gender = nem.diff('he', 'she')

In [48]:
nem

In [49]:
# analogies gender
a_gender = nem.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 1998.0
Median: 1998.0
he-she
going-bad
listening-united
became-frowning
worth-insecure
despair-lord
brother-white
sing-death
gt-ugly
religion-miserable
match-king
hurting-switch
can-anyone
mood-grow
8-anything
expecting-legs
100-ex
daily-nation
favorite-hour
houstontexans-compliment
tonight-give
door-india
straight-mini
bitter-mean
madden-sweet
piss-by
guess-motivation
10-hair
shudder-server
up-put
gives-an
bf-lit
reply-low
draw-hit
vs-chance
spent-7
sending-faced
burst-mom
say-hope
sulk-window
forward-40
changes-bees
protests-lil
pray-arsehole
movie-experience
aggravate-security
mention-huge
harm-fuckin
sadness-horrific
disappointment-does
likes-down
wake-exhilarating
terrific-facewithtearsofjoyfacewithtearsofjoy
wasted-behappy
complete-beyond
oil-wine
huh-temper
collect-neutralface
treat-hi
half-tomorrow
totally-midnight
fucked-held
centre-spirit
joke-worried
don-posted
posts-ghost
said-literally
others-glee
election-wasnt
voice-those
coach-subway
terrorists

In [50]:
sp = sorted([(nem.v(w).dot(v_gender), w) for w in res_i])
sp[0:20], sp[-20:]

([(0.0007325481, 'dad'),
  (0.0014068095, 'student'),
  (0.00357544, 'character'),
  (0.0046673967, 'president'),
  (0.0048077074, 'writer'),
  (0.0050418265, 'doctor'),
  (0.0058117495, 'author'),
  (0.0071301213, 'teacher'),
  (0.008055753, 'coach')],
 [(0.0007325481, 'dad'),
  (0.0014068095, 'student'),
  (0.00357544, 'character'),
  (0.0046673967, 'president'),
  (0.0048077074, 'writer'),
  (0.0050418265, 'doctor'),
  (0.0058117495, 'author'),
  (0.0071301213, 'teacher'),
  (0.008055753, 'coach')])

In [51]:
%cd ..
%cd data

/content/debiaswe
/content/debiaswe/data


In [52]:
import json

In [78]:
# Lets load some gender related word lists to help us with debiasing
with open('definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['her', 'his']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [77]:
!pwd
%cd data

/content/debiaswe
/content/debiaswe/data


In [79]:
%cd ..
%cd debiaswe

/content/debiaswe
/content/debiaswe/debiaswe


In [80]:
!ls
%cd ..

data.py    embeddingtext1.txt  learn_gender_specific.py  we.py
debias.py  __init__.py	       __pycache__
/content/debiaswe


In [70]:
# import errors, sourced from debiaswe/debiaswe/debias

def debias(embedding, gender_specific_words, definitional, equalize):
    E = embedding
    gender_direction = we.doPCA(definitional, E).components_[0]
    specific_set = set(gender_specific_words)
    for i, w in enumerate(E.words):
        if w not in specific_set:
            E.vecs[i] = we.drop(E.vecs[i], gender_direction)
    E.normalize()
    candidates = {x for e1, e2 in equalize for x in [(e1.lower(), e2.lower()),
                                                     (e1.title(), e2.title()),
                                                     (e1.upper(), e2.upper())]}
    print(candidates)
    for (a, b) in candidates:
        if (a in E.index and b in E.index):
            y = we.drop((E.v(a) + E.v(b)) / 2, gender_direction)
            z = np.sqrt(1 - np.linalg.norm(y)**2)
            if (E.v(a) - E.v(b)).dot(gender_direction) < 0:
                z = -z
            E.vecs[E.index[a]] = z * gender_direction + y
            E.vecs[E.index[b]] = -z * gender_direction + y
    E.normalize()
    return E



In [81]:
nem_db = debias(nem, gender_specific_words, defs, equalize_pairs)

1999 words of dimension 300 : i, im, like, amp, ..., strike, cruel, michael, arsehole
{('Uncle', 'Aunt'), ('HE', 'SHE'), ('gelding', 'mare'), ('fraternity', 'sorority'), ('MEN', 'WOMEN'), ('males', 'females'), ('BROTHERS', 'SISTERS'), ('FATHER', 'MOTHER'), ('Male', 'Female'), ('gentlemen', 'ladies'), ('GRANDFATHER', 'GRANDMOTHER'), ('chairman', 'chairwoman'), ('congressman', 'congresswoman'), ('Fraternity', 'Sorority'), ('BUSINESSMAN', 'BUSINESSWOMAN'), ('brothers', 'sisters'), ('GENTLEMAN', 'LADY'), ('Father', 'Mother'), ('CONGRESSMAN', 'CONGRESSWOMAN'), ('twin_brother', 'twin_sister'), ('SPOKESMAN', 'SPOKESWOMAN'), ('COUNCILMAN', 'COUNCILWOMAN'), ('Fella', 'Granny'), ('Kings', 'Queens'), ('sons', 'daughters'), ('Dudes', 'Gals'), ('TESTOSTERONE', 'ESTROGEN'), ('Wives', 'Husbands'), ('Catholic_Priest', 'Nun'), ('prince', 'princess'), ('KING', 'QUEEN'), ('EX_GIRLFRIEND', 'EX_BOYFRIEND'), ('Schoolboy', 'Schoolgirl'), ('GRANDPA', 'GRANDMA'), ('fella', 'granny'), ('Gelding', 'Mare'), ('HIS